# Introduction
Handling materials within BOM Analysis brings together a number of different topics within the examples.

All components have a default material class. This default MaterialData does not have a data repository but acts as a parent to such a repository.

In [ ]:
from bom_analysis import Component, Assembly

breeder = Component(ref="breeder")
breeder.material

There are two material classes in-built to BOM Analysis which can be used. 

1. DFLibraryWrap imports data into a dataframe from a .json
2. CoolPropsWrap wraps CoolProps

_Note, DFLibraryWrap needs a path to the .json_

In [ ]:
from bom_analysis.materials import DFLibraryWrap, CoolPropsWrap

eurofer = DFLibraryWrap(mat="steel")
eurofer.path = "./files/example_material_properties.json"

helium = CoolPropsWrap(mat="He")

### data_wrapper() - Getting Material Data
In the loaded example, a MaterialData child which loads data from a .json has been used. It allows the data_wrapper method to extract the material properties.

Note that the material defaults to ambient pressure and room temperature.

In [ ]:
eurofer.data_wrapper("thermal_conductivity")

In [ ]:
helium.data_wrapper("conductivity")

From this example, some of the challenges with different naming conventions can be seen. Importing and populating the translator aims to help with this.

In [ ]:
try:
    helium.data_wrapper("thermal_conductivity")
except:
    pass

In [ ]:
from bom_analysis.utils import Translator

Translator.define_translations(["./files/translation.json"])
helium.data_wrapper("thermal_conductivity")

Within the errors in the above try-except the materials priority is mentioned. This is an attribute of the Config file that sets a priority order for the classes which make up the materials.

## Materials Selector
The Config.materials is a materials selector class which can be loaded as part of the configuration. For this example, it will be considered as a standalone.

Libraries and information can be added to the materials selector in an order of priority,


In [ ]:
from bom_analysis.utils import MaterialSelector

ms = MaterialSelector()
ms.add_database(DFLibraryWrap, {"path": "./files/example_material_properties.json"})
ms.add_database(CoolPropsWrap)
steel = ms.select_database("steel")
He = ms.select_database("He")

The selector returns a class for the particular material database.

In [ ]:
print(steel, "\n", He)
steel.data_wrapper("thermal_conductivity")

This allows components to be assigned materials directly, however, BOM Analysis allows for improved functionality by using the Config.material

## Material Selector in Config
Things become more automated when assigning the material selector to the config as all classes have access to the material selector

In [ ]:
from bom_analysis.base import BaseConfig as Config

Config.materials = ms

All classes now have access to the material selector. One of the benefits is when a material property does not exist in on library, BOM analysis will check the others. This has an overhead but can be useful for particularly uncommon properties. Below, the example materials has the thermal conductivity for CO2 but nothing else, so when asked for density, CoolProps will be checked.

In [ ]:
co2 = Config.materials.select_database("CarbonDioxide")
print(ms.priority_order)
print(co2.data_wrapper("thermal_conductivity"))
print(co2.data_wrapper("density"))

The errors occur because it was unable to find the data in the DFLibrary but a value is found in the CoolProps library

Finaly, with the config material selector assigned all components can be assigned material properties automatically (if data exists).

In [ ]:
from bom_analysis import Q_

breeding_zone = Assembly(ref="bz")
coolant = Component(ref="coolant")
coolant.material.mat = "He"
coolant.material.pressure = Q_(80, "bar")
structure = Component(ref="structure")
structure.material.mat = "steel"
structure.material.reftemp = Q_(500, "degC")
breeding_zone.add_components([coolant, structure])
breeding_zone.assign_all_materials()
print(structure.material)
print(coolant.material)

## Conclusion
The material data can be handled automatically with the above classes and methods allowing for easy data selection.